In [1]:
import datetime

import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from scipy.sparse import csr_matrix, hstack, coo_matrix

import string
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

from nltk.stem.snowball import EnglishStemmer
import nltk

import pickle

from sklearn.decomposition import LatentDirichletAllocation

In [2]:
business = pd.read_json('yelp_dataset/business.json', lines=True)
#checkin = pd.read_json('yelp_dataset/checkin.json', lines=True)
#photo = pd.read_json('yelp_dataset/photo.json', lines=True)
#review =  pd.read_json('yelp_dataset/review.json', lines=True)
#tip = pd.read_json('yelp_dataset/tip.json', lines=True)
##user = pd.read_json('yelp_dataset/user.json', lines=True)

In [3]:
business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


## Helper Function

In [ ]:
def text_process(text):
    """
    Modified from
    http://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/
    Takes in a string of text, then performs the following:
    1. Remove all punctuation, and digits 
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    stemmer = EnglishStemmer()
   
    # Check characters to see if they are in punctuation
    clean = [char for char in text if (char not in string.punctuation) 
            and (not char.isdigit())] 
 
    clean = ''.join(clean)
    tokens = clean.split()
    tokens = [stemmer.stem(c) for c in tokens]
    # Join the characters again to form the string.

    tokens = ' '.join(tokens)
    
    # Now just remove any stopwords
    return tokens

In [ ]:
#https://github.com/pandas-dev/pandas/issues/18152

max_records = 1e5
df = pd.read_json('yelp_dataset/review.json', lines=True, chunksize=max_records)
reviews = pd.DataFrame() # Initialize the dataframe
try:
   for df_chunk in df:
       reviews = pd.concat([reviews, df_chunk])
except ValueError:
       print ('\nSome messages in the file cannot be parsed')

In [ ]:
print(reviews.shape)
reviews.head()

In [ ]:
joined = reviews.join(business, lsuffix='_r', rsuffix='_b', how="inner")

In [ ]:
print(joined.shape)
joined.head()

#### Filter restaurants

In [ ]:
temp = joined['categories'].str.contains('Restaurants', regex=False)
temp = temp.fillna(False)
joined = joined[temp]
print(joined.shape)
joined.head()

### get city list

In [ ]:
joined['city'].value_counts(sort=True)

### Get categories

In [ ]:
joined['categories'].value_counts(sort=True)

### Good/bad breakdown for given categories

In [ ]:
common_cats = ['Pizza', 'Mexican', 'Chinese', 'Italian', 'Vietnamese']

#### Pizza

In [ ]:
temp = joined['categories'].str.contains('Pizza', regex=False)
temp = temp.fillna(False)
pizza = joined[temp]
pizza.loc[pizza['stars_r'] > 3, 'review_type'] = "good"
pizza.loc[pizza['stars_r'] <= 3, 'review_type'] = "bad"
p_types = pizza['review_type'].value_counts()

#### Mexican

In [ ]:
temp = joined['categories'].str.contains('Mexican', regex=False)
temp = temp.fillna(False)
mexican = joined[temp]
mexican.loc[mexican['stars_r'] > 3, 'review_type'] = "good"
mexican.loc[mexican['stars_r'] <= 3, 'review_type'] = "bad"
m_types = mexican['review_type'].value_counts()

#### Chinese

In [ ]:
temp = joined['categories'].str.contains('Chinese', regex=False)
temp = temp.fillna(False)
chinese = joined[temp]
chinese.loc[chinese['stars_r'] > 3, 'review_type'] = "good"
chinese.loc[chinese['stars_r'] <= 3, 'review_type'] = "bad"
c_types = chinese['review_type'].value_counts()

#### Italian

In [ ]:
temp = joined['categories'].str.contains('Italian', regex=False)
temp = temp.fillna(False)
italian = joined[temp]
italian.loc[italian['stars_r'] > 3, 'review_type'] = "good"
italian.loc[italian['stars_r'] <= 3, 'review_type'] = "bad"
i_types = italian['review_type'].value_counts()

#### Vietnamese

In [ ]:
temp = joined['categories'].str.contains('Vietnamese', regex=False)
temp = temp.fillna(False)
vietnamese = joined[temp]
vietnamese.loc[vietnamese['stars_r'] > 3, 'review_type'] = "good"
vietnamese.loc[vietnamese['stars_r'] <= 3, 'review_type'] = "bad"
v_types = vietnamese['review_type'].value_counts()

In [ ]:
p_types

In [ ]:
p_types[1]

In [ ]:
m_types

In [ ]:
d = {'bad': [p_types[1], m_types[1], c_types[1], i_types[1], v_types[1]], 'good': [p_types[0], m_types[0], c_types[0], i_types[0], v_types[0]]}
review_types = pd.DataFrame(data=d, index=['Pizza', 'Mexican', 'Chinese', 'Italian', 'Vietnamese'])
review_types

In [ ]:
review_types['index'] = review_types.index
review_types.to_csv("static/data/review_types.csv", sep=',')


### Good/bad breakdown for given cities

#### Las Vegas

In [ ]:
lv = joined[joined['city'] == "Las Vegas"]
lv.loc[lv['stars_r'] > 3, 'review_type'] = "good"
lv.loc[lv['stars_r'] <= 3, 'review_type'] = "bad"
lv_types = lv['review_type'].value_counts()
lv_types

#### Phoenix

In [ ]:
ph = joined[joined['city'] == "Phoenix"]
ph.loc[ph['stars_r'] > 3, 'review_type'] = "good"
ph.loc[ph['stars_r'] <= 3, 'review_type'] = "bad"
ph_types = ph['review_type'].value_counts()
ph_types

#### Madison

In [ ]:
mad = joined[joined['city'] == "Madison"]
mad.loc[mad['stars_r'] > 3, 'review_type'] = "good"
mad.loc[mad['stars_r'] <= 3, 'review_type'] = "bad"
mad_types = mad['review_type'].value_counts()
mad_types

#### Pittsburgh

In [ ]:
pit = joined[joined['city'] == "Pittsburgh"]
pit.loc[pit['stars_r'] > 3, 'review_type'] = "good"
pit.loc[pit['stars_r'] <= 3, 'review_type'] = "bad"
pit_types = pit['review_type'].value_counts()
pit_types

#### Charlotte

In [ ]:
cha = joined[joined['city'] == "Charlotte"]
cha.loc[cha['stars_r'] > 3, 'review_type'] = "good"
cha.loc[cha['stars_r'] <= 3, 'review_type'] = "bad"
cha_types = cha['review_type'].value_counts()
cha_types

In [ ]:
d = {'bad': [lv_types[1], ph_types[1], mad_types[1], pit_types[1], cha_types[1]], 'good': [lv_types[0], ph_types[0], mad_types[0], pit_types[0], cha_types[0]]}
city_review_types = pd.DataFrame(data=d, index=['Las Vegas', 'Phoenix', 'Madison', 'Pittsburgh', 'Charlotte'])
city_review_types

In [ ]:
city_review_types['index'] = city_review_types.index
city_review_types.to_csv("static/data/city_review_types.csv", sep=',')


In [ ]:
#https://www.w3schools.com/python/ref_string_join.asp
def tokenize(string):
    d = []
    tokens = nltk.word_tokenize(string)
    tags = nltk.pos_tag(tokens)
    for x in tags:
        if x[1] == "NN" or x[1] == "NNS" or x[1] == "NNP" or x[1] == "NNPS":
            d.append(x[0])
    d = " ".join(d)
    return d

### Train/test split

In [ ]:
temp = joined[['text', 'city', 'categories']]
X_train, X_test, y_train, y_test = train_test_split(temp, joined['stars_r'], test_size=0.2, random_state=42)
X_train.head()

### TF-IDF

In [ ]:
corpus = X_train['text']
vectorizer = TfidfVectorizer(min_df = 0.01)

vectorizer.fit(corpus)

tf_train = vectorizer.transform(corpus)
tf_train

In [ ]:
#test data
test_corpus = X_test['text']
tf_test = vectorizer.transform(test_corpus)
tf_test

tf_train = pd.SparseDataFrame(tf_train.toarray(),columns=vectorizer.get_feature_names(),index=X_trainC.index)
tf_test = pd.SparseDataFrame(tf_test.toarray(),columns=vectorizer.get_feature_names(),index=X_testC.index)
tf_test.head()

### One-hot encoding

In [ ]:
X_train.head()

In [ ]:
#common_cats = ['Nightlife', 'Pizza', 'Burger', 'Chinese', 'Steak', 'Sandwiches', 'Fast Food']
common_cats = ['Pizza', 'Mexican', 'Chinese', 'Italian', 'Vietnamese']

In [ ]:
for cat in common_cats:
    X_train[cat] = 0
    X_test[cat] = 0
X_train.head()

In [ ]:
#https://stackoverflow.com/questions/36909977/update-row-values-where-certain-condition-is-met-in-pandas
for cat in common_cats:
    X_train.loc[X_train['categories'].str.contains(cat, regex=False), [cat]] = 1
    X_test.loc[X_test['categories'].str.contains(cat, regex=False), [cat]] = 1
X_train.head()

In [ ]:
X_train['city'] = X_train['city'].str.lower()
X_test['city'] = X_test['city'].str.lower()

#X_train['state'] = X_train['state'].str.lower()
#X_test['state'] = X_test['state'].str.lower()

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train['city'].reshape(-1,1))

In [ ]:
X_trainC = enc.transform(X_train['city'].reshape(-1,1))
X_testC = enc.transform(X_test['city'].reshape(-1,1))
X_trainC

In [ ]:
cities = []
for t in enc.categories_:
    for c in t:
        cities.append(c)

In [ ]:
city_df = pd.SparseDataFrame(X_trainC, columns=cities)
list(city_df)

enc.inverse_transform([["phoenix"]])

In [ ]:
X_train[common_cats].values

In [ ]:
X_train_cats = csr_matrix(X_train[common_cats].values) 
X_test_cats = csr_matrix(X_test[common_cats].values)
X_train_cats

In [ ]:
def genCity(city):
    row = pd.DataFrame(columns=cities)
    row.loc[0] = [0]*len(cities)
    row.loc[:, city] = 1
    row = row.astype('int64')
    row = csr_matrix(row.values)
    return row

In [ ]:
genCity("agincourt")

### Join results

In [ ]:
X_trainJ = hstack([tf_train, X_trainC, X_train_cats], format="csr")
X_testJ = hstack([tf_test, X_testC, X_test_cats], format="csr")

### Grid Search

#### Ridge

In [ ]:
parameters = {"alpha":[0.0001, 0.001, 0.01, 0.1]}

In [ ]:
ridge = Ridge()
ridge_grid = GridSearchCV(estimator=ridge, param_grid=parameters, cv=10, scoring="r2", n_jobs=20)
print(ridge_grid)

In [ ]:
print(X_train.shape)
print(y_train.shape)
ridge_grid.fit(X_trainJ, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(ridge_grid.best_params_)
print()
print("Grid scores on development set:")
print()
means = ridge_grid.cv_results_['mean_test_score']
stds = ridge_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, ridge_grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

#### Lasso

In [ ]:
lasso = Lasso()

In [ ]:
lasso_grid = GridSearchCV(estimator=lasso, param_grid=parameters, cv=10, scoring="r2", n_jobs=20)
print(lasso_grid)

In [ ]:
lasso_grid.fit(X_trainJ, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(lasso_grid.best_params_)
print()
print("Grid scores on development set:")
print()
means = lasso_grid.cv_results_['mean_test_score']
stds = lasso_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, lasso_grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

#### Elastic Net

In [ ]:
ElasticNet = ElasticNet()

In [ ]:
en_grid = GridSearchCV(estimator=ElasticNet, param_grid=parameters, cv=10, scoring="r2", n_jobs=20)
print(en_grid)

In [ ]:
en_grid.fit(X_trainJ, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(en_grid.best_params_)
print()
print("Grid scores on development set:")
print()
means = en_grid.cv_results_['mean_test_score']
stds = en_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, en_grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

### Predict outputs

In [ ]:
en_final = ElasticNet(alpha=0.0001)
en_final.fit(X_trainJ, y_train)


In [ ]:
y_pred = en_final.predict(X_testJ)
y_pred

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
def genCat(cat, common_cats):
    row = pd.DataFrame(columns=common_cats)
    row.loc[0] = [0]*len(common_cats)
    row.loc[:, cat] = 1
    row = row.astype('int64')
    row = csr_matrix(row.values)
    return row

In [ ]:
def get_size(text, city, cat):

    text_tf = vectorizer.transform(text)
    text_enc = genCity(city)
    cat_row = genCat(cat, common_cats)

    text_joined = hstack([text_tf, text_enc, cat_row], format="csr")
    return text_joined

In [ ]:
get_size(["this is a review"], "las vegas", "Pizza")

In [ ]:
len(common_cats)

def predict(text):

    text_tf = vectorizer.transform(text)

    #text_lower = text.str.lower()
    text_lower = text[0].lower()

    #text_enc = enc.transform(text_lower.reshape(-1,1))
    text_enc = enc.transform([[text_lower]])

    text_joined = hstack([text_tf, text_enc], format="csr")
    return en_final.predict(text_joined)

def predict_city(text, city):

    text_tf = vectorizer.transform(text)
    text_enc = genCity(city)

    text_joined = hstack([text_tf, text_enc], format="csr")
    return en_final.predict(text_joined)

In [ ]:
def predict_city_cat(text, city, cat):

    text_tf = vectorizer.transform(text)
    text_enc = genCity(city)
    cat_row = genCat(cat, common_cats)

    text_joined = hstack([text_tf, text_enc, cat_row], format="csr")
    return en_final.predict(text_joined)

print(predict(["this is a review"]))
print(predict(["This is a great restaurant!"]))
print(predict(["This is a bad restaurant!"]))

print(predict_city(["this is a review"], "phoenix"))
print(predict_city(["this is a review"], "las vegas"))
#print(predict_city(["this is a review"], "boston"))
print()
print(predict_city(["This is a great restaurant!"], "phoenix"))
print(predict_city(["This is a great restaurant!"], "las vegas"))
#print(predict_city(["This is a great restaurant!"], "boston"))
print()
print(predict_city(["This is a bad restaurant!"], "phoenix"))
print(predict_city(["This is a bad restaurant!"], "las vegas"))
#print(predict_city(["This is a bad restaurant!"], "boston"))


In [ ]:
print(predict_city_cat(["this is a review"], "phoenix", "Pizza"))
print(predict_city_cat(["this is a review"], "las vegas", "Pizza"))
print(predict_city_cat(["this is a review"], "phoenix", "Chinese"))
print(predict_city_cat(["this is a review"], "las vegas", "Chinese"))
print()
print(predict_city_cat(["This is a great restaurant!"], "phoenix", "Pizza"))
print(predict_city_cat(["This is a great restaurant!"], "las vegas", "Pizza"))
print(predict_city_cat(["This is a great restaurant!"], "phoenix", "Chinese"))
print(predict_city_cat(["This is a great restaurant!"], "las vegas", "Chinese"))
print(predict_city_cat(["This is a good review"], "las vegas", "Chinese"))
print()
print(predict_city_cat(["This is a bad restaurant!"], "phoenix", "Pizza"))
print(predict_city_cat(["This is a bad restaurant!"], "las vegas", "Pizza"))
print(predict_city_cat(["This is a bad restaurant!"], "phoenix", "Chinese"))
print(predict_city_cat(["This is a bad restaurant!"], "las vegas", "Chinese"))


### pickle

In [ ]:
with open('enc.pkl', 'wb') as enc_file:
    pickle.dump(enc, enc_file)

with open('vec.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('model.pkl', 'wb') as file:
    pickle.dump(en_final, file)

In [ ]:
s = pickle.dumps(en_final)
p = pickle.loads(s)

In [ ]:
def pickle_predict(text, city, cat):

    text_tf = vectorizer.transform(text)
    text_enc = genCity(city)
    cat_row = genCat(cat, common_cats)

    text_joined = hstack([text_tf, text_enc, cat_row], format="csr")
    return p.predict(text_joined)

In [ ]:
print(pickle_predict(["this is a review"], "phoenix", "Pizza"))
print(pickle_predict(["this is a review"], "las vegas", "Pizza"))
print(pickle_predict(["this is a review"], "phoenix", "Chinese"))
print(pickle_predict(["this is a review"], "las vegas", "Chinese"))
print()
print(pickle_predict(["This is a great restaurant!"], "phoenix", "Pizza"))
print(pickle_predict(["This is a great restaurant!"], "las vegas", "Pizza"))
print(pickle_predict(["This is a great restaurant!"], "phoenix", "Chinese"))
print(pickle_predict(["This is a great restaurant!"], "las vegas", "Chinese"))
print(pickle_predict(["This is a good review"], "las vegas", "Chinese"))
print()
print(pickle_predict(["This is a bad restaurant!"], "phoenix", "Pizza"))
print(pickle_predict(["This is a bad restaurant!"], "las vegas", "Pizza"))
print(pickle_predict(["This is a bad restaurant!"], "phoenix", "Chinese"))
print(pickle_predict(["This is a bad restaurant!"], "las vegas", "Chinese"))

In [ ]:
X_trainJ

In [ ]:
X_trainJS = pd.SparseDataFrame(X_trainJ)
X_trainJS.head()